In [ ]:
!ls /kaggle/working/

25461.png	   cr1.pt			  mean_comparison_plot.png
28076.png	   cr2.pt			  mr.pt
94162.png	   final_model_concept.pt	  rr.npy
96704.png	   final_model_concept_inlp.pt	  rr1.npy
causal_vb.pt	   final_model_wo_adversarial.pt  transformers
causal_vb_inlp.pt  foo1.png			  vs_tensors.pt
cr.npy		   full_annotation.pkl		  y_p.pt
cr.pt		   image1.jpg
cr1.npy		   image2.jpg


In [ ]:
#RUN

import ipywidgets as widgets



In [ ]:
SEED = 42
FILE = '/kaggle/working/final_model_concept.pt'

In [ ]:
#RUN
ff = []

In [ ]:
#RUN
!pip install jsonlines


In [ ]:
#RUN
!pip install captum==0.7.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 22.9 MB/s eta 0:00:00a 0:00:01


In [ ]:
#R
import jsonlines

In [ ]:
#R
!pip install wget
#!pip install transformers

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=1db1ceb5231a559a8ef8eccb0bca9cb3e2a75ee1f70417670fb7b61a1284b583
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [ ]:
#R
import sys
sys.path.append('/kaggle/working/transformers/examples/research_projects/visual_bert')

In [ ]:
#R
import numpy as np
import jsonlines
from tqdm import tqdm
import os
import sys
import os
from IPython.display import Image, display
import PIL.Image
import io
import torch
import numpy as np
from processing_image import Preprocess
from visualizing_image import SingleImageViz
from modeling_frcnn import GeneralizedRCNN
from utils import Config
import utils
from transformers import VisualBertForQuestionAnswering, BertTokenizerFast
import pandas as pd
import random
from sklearn.model_selection import cross_val_score, cross_val_predict, KFold
from sklearn import svm
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [ ]:
#R
train_file = '/kaggle/input/facebook-hateful-meme-dataset/data/train.jsonl'
k =0
with jsonlines.open(train_file) as reader:
    for obj in reader:
        print(obj['img'])
        if k==9:
            break
        k+=1

img/42953.png
img/23058.png
img/13894.png
img/37408.png
img/82403.png
img/16952.png
img/76932.png
img/70914.png
img/02973.png
img/58306.png


In [ ]:
#R
visual_feats = {}

In [ ]:
#R
device = 'cuda'
frcnn_cfg = Config.from_pretrained("unc-nlp/frcnn-vg-finetuned")
frcnn_cfg.MODEL.device = device
frcnn = GeneralizedRCNN.from_pretrained("unc-nlp/frcnn-vg-finetuned", config=frcnn_cfg)
image_preprocess = Preprocess(frcnn_cfg)
frcnn.eval()



%s not found in cache or force_download set to True, downloading to %s https://s3.amazonaws.com/models.huggingface.co/bert/unc-nlp/frcnn-vg-finetuned/config.yaml /root/.cache/torch/transformers/tmpjdrzlwm3


Downloading:   0%|          | 0.00/2.13k [00:00<?, ?B/s]

loading configuration file cache
%s not found in cache or force_download set to True, downloading to %s https://cdn.huggingface.co/unc-nlp/frcnn-vg-finetuned/pytorch_model.bin /root/.cache/torch/transformers/tmpdp33_yuo


Downloading:   0%|          | 0.00/262M [00:00<?, ?B/s]

loading weights file https://cdn.huggingface.co/unc-nlp/frcnn-vg-finetuned/pytorch_model.bin from cache at /root/.cache/torch/transformers/57f6df6abe353be2773f2700159c65615babf39ab5b48114d2b49267672ae10f.77b59256a4cf8343ae0f923246a81489fc8d82f98d082edc2d2037c977c0d9d0
All model checkpoint weights were used when initializing GeneralizedRCNN.

All the weights of GeneralizedRCNN were initialized from the model checkpoint at unc-nlp/frcnn-vg-finetuned.
If your task is similar to the task the model of the checkpoint was trained on, you can already use GeneralizedRCNN for predictions without further training.


GeneralizedRCNN(
  (backbone): ResNet(
    (stem): BasicStem(
      (conv1): Conv2d(
        3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
        (norm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (res2): Sequential(
      (0): BottleneckBlock(
        (shortcut): Conv2d(
          64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False
          (norm): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (conv1): Conv2d(
          64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False
          (norm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (conv2): Conv2d(
          64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
          (norm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (conv3): Conv2d(
          64, 256, kernel_size=(1, 1), 

In [ ]:
#R
def get_visual_embedding(img_paths):
    images, sizes, scales_yx = image_preprocess(img_paths) # img_paths -> list of image paths
    output_dict = frcnn(
      images,
      sizes,
      scales_yx=scales_yx,
      padding="max_detections",
      max_detections=frcnn_cfg.max_detections,
      return_tensors="pt",
    )
    features = output_dict.get("roi_features")
    visual_embeds = features
    visual_token_type_ids = torch.ones(visual_embeds.shape[:-1], dtype=torch.long)
    visual_attention_mask = torch.ones(visual_embeds.shape[:-1], dtype=torch.float)
    return visual_embeds,visual_token_type_ids,visual_attention_mask

In [ ]:
#R
visual_feats = torch.load('./vs_tensors.pt')

In [ ]:
#R
len(visual_feats)

8500

In [ ]:
# visual_feats

In [ ]:
#R
import torch
from torch.nn import functional as F
import numpy as np
from utils import Config
import utils
from transformers import VisualBertModel, BertTokenizer
import pandas as pd
import pickle
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.metrics import accuracy_score,f1_score,roc_auc_score
from scipy.special import softmax

In [ ]:
#R
train_set = []
train_file = '/kaggle/input/facebook-hateful-meme-dataset/data/train.jsonl'
k =0
with jsonlines.open(train_file) as reader:
    for obj in reader:
    #         print(obj['img'])

        train_set.append({'text': obj['text'],

                       'img': obj['img'],

                       'label': int(obj['label'])})
    #         print(k)
    #         if k==1500:
    #             break
        k+=1



In [ ]:
#R
len(train_set)

8500

In [ ]:
#R
class CustomDataset(Dataset):

    def __init__(self, ds, max_len=64):
        self.ds = ds
        self.tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
        self.model = VisualBertModel.from_pretrained("uclanlp/visualbert-vqa-coco-pre")


        self.max_len = max_len
    def __len__(self):
        return len(self.ds)

    def __getitem__(self, index):

        ds_idx = self.ds[index]
        inputs = self.tokenizer(ds_idx['text'], padding="max_length", truncation=True, max_length=self.max_len, return_tensors='pt')


        inputs['input_ids'] = inputs['input_ids'].squeeze(0)
        inputs['token_type_ids'] = inputs['token_type_ids'].squeeze(0)
        inputs['attention_mask'] = inputs['attention_mask'].squeeze(0)

        # visual_embeds,visual_token_type_ids,visual_attention_mask = get_visual_embedding('./data/'+ds_idx['img'])
        visual_embeds,visual_token_type_ids,visual_attention_mask = visual_feats[ds_idx['img']]

        #print(ds_idx['img'])

        inputs.update({
          "visual_embeds": torch.squeeze(visual_embeds),
          "visual_token_type_ids": torch.squeeze(visual_token_type_ids),
          "visual_attention_mask": torch.squeeze(visual_attention_mask)
        })



        #inputs.update({"path": ds_idx['img']})

        return inputs, int(ds_idx['label'])

t = CustomDataset(train_set)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/631 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/448M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [ ]:
#!export CUBLAS_WORKSPACE_CONFIG=:16:8

In [ ]:
#R
seed=SEED
random.seed(seed)     # python random generator
np.random.seed(seed)  # numpy random generator

torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
#torch.use_deterministic_algorithms(True)

In [ ]:
len(t)

8500

In [ ]:
#R
torch.manual_seed(42)
t_p,te_p = torch.utils.data.random_split(t,[7840,660])

In [ ]:
#R
from torch.utils.data import DataLoader

train_dataloader = DataLoader(t_p, shuffle=True, batch_size=32)
eval_dataloader = DataLoader(te_p, batch_size=1)

In [ ]:
#R
from PIL import Image
import matplotlib.pyplot as plt

In [ ]:
print('h')

h


In [ ]:
#R
concept_classes = ["holocaust", "nazism", "genocide", "funny", "anti muslim", "terrorism", "violence", "politics", "racism", "international relation", "adult", "gore", "misogynistic", "immigration", "extremism", "immoral", "white supremacy", "indecency"]


In [ ]:
len(concept_classes)

18

In [ ]:
#cnt

In [ ]:
#R
with open('/kaggle/working/full_annotation.pkl', 'rb') as f:
    mynewlist = pickle.load(f)

In [ ]:
len(mynewlist)

208

In [ ]:
#R
gc=mynewlist

In [ ]:
#R
device='cuda'

In [ ]:
device

'cuda'

In [ ]:
#R
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")


In [ ]:
tokenizer.encode("jew")[1:-1]

[16522]

In [ ]:
#R
DYN_ROUTE = True
DECONFOUND = True

In [ ]:
#R
!pip install pytorch_revgrad


In [ ]:
#torch.nn.Parameter(torch.tensor(0.1))

In [ ]:
#R
num_concepts = 18
#BS = 8
import torch
from torch import nn
from pytorch_revgrad import RevGrad
class VB(nn.Module):
    def __init__(self):
        super(VB, self).__init__()
        self.vb = VisualBertModel.from_pretrained("uclanlp/visualbert-vqa-coco-pre")

        self.linear_relu_stack = nn.Sequential(
            nn.Dropout(0.1),
            nn.Linear(768, 512),
            nn.ReLU(),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Linear(128, 2),
        )


        self.linear_relu_stack_latent = nn.Sequential(
            nn.Dropout(0.1),
            nn.Linear(768, 512),
            nn.ReLU(),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Linear(128, 2),
        )


        self.linear_relu_stack_hidden = nn.Sequential(
            nn.Dropout(0.1),
            nn.Linear(768, 512),
            nn.ReLU(),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Linear(128, 2),
        )

        #self.w = torch.nn.Parameter(torch.ones(11,1)/11)
        self.w = torch.nn.Parameter(torch.ones(num_concepts,1))

        self.project_latent = nn.Linear(768,2048)

        #self.W_ij = [[nn.Linear(768,768) for i in range(64)] for j in range(17)]

        self.W_ij = torch.nn.Parameter(torch.randn(18,64,28,28))

        self.proj_conc = nn.Linear(768,28)
        self.proj_embed = nn.Linear(768,28)



        self.grad_rev1 = RevGrad()
        self.grad_rev2 = RevGrad()

        self.w1 = torch.nn.Parameter(torch.tensor(0.2))


    def forward(self,
        all_concepts_,
        input_ids = None,
        attention_mask = None,
        token_type_ids = None,
        position_ids = None,
        head_mask = None,
        inputs_embeds = None,
        visual_embeds = None,
        visual_attention_mask = None,
        visual_token_type_ids = None,
        image_text_alignment = None,
        output_attentions = None,
        output_hidden_states = None,
        return_dict = None):

        #         print(all_concepts.shape)

        B = visual_embeds.size(0)

        #         print(B)




        ##############################################

        all_concepts = self.proj_conc(all_concepts_)



        with torch.no_grad():
            inputs_embeds = self.vb.embeddings.word_embeddings(input_ids)

        #print(0/0)

        inputs_embeds = self.proj_embed(inputs_embeds)


        #####################################################





        #b1 = torch.bmm(inputs_embeds,all_concepts.repeat(B,1,1).transpose(1,2))

        #print(inputs_embeds.shape)
        #print(all_concepts.repeat(B,1,1).transpose(1,2).shape)

        #b,64,28
        #b,28,17


        b = torch.einsum('blx,bxn->bln',inputs_embeds,all_concepts.repeat(B,1,1).transpose(1,2))

        #         print(b1)

        #         print('********')

        #         print(b2)

        #         print(torch.equal(b1,b2))



        b = b.masked_fill(~attention_mask.unsqueeze(dim=2).bool(), -np.inf)
        c = F.softmax(b,dim=1)



        #inter = torch.einsum('bcsn,csnn->bcsn',inputs_embeds.unsqueeze(1).repeat(1,17,1,1), self.W_ij)

        #print(inter[:,0,0,:])

        einsum_expression = 'bln,clnx->bclx'
        inter = torch.einsum(einsum_expression, inputs_embeds, self.W_ij)

        k = torch.einsum('bcsn,bsc->bcn',inter,c)
        norm = torch.norm(k,dim=2)



        kk1 = ((norm**2)/(1+norm**2)).unsqueeze(dim=2) * (k/norm.unsqueeze(dim=2))

        w_dynamic = kk1.norm(dim=2).unsqueeze(2)


        raw_latent = (1-self.w1) * all_concepts_.unsqueeze(0).repeat(B,1,1) * self.w.unsqueeze(dim=0).repeat(B,1,1) + all_concepts_.unsqueeze(0).repeat(B,1,1) * w_dynamic * self.w1

        raw_latent = torch.mean(raw_latent, dim=1)

        #raw_latent = torch.mean(all_concepts*self.w,dim=0)
        #raw_latent = raw_latent.unsqueeze(dim=0)

        latent = self.project_latent(raw_latent)



        latent = latent.unsqueeze(dim=1)




        visual_embeds_latent = torch.cat((visual_embeds,latent),dim=1)


        ################################################################


        #print(b)

        #######################################################
        #         print(b.shape)

        #         b = []
        #         for i in range(64):
        #             bi = []
        #             for j in range(17):
        #                 #print(inputs_embeds[:,i,:].shape)
        #                 ci = all_concepts[j].unsqueeze(0).repeat(32,1,1).squeeze()
        #                 uj = inputs_embeds[:,i,:]
        #                 #print(ci.shape)
        #                 bij = torch.bmm(uj.unsqueeze(dim=1), ci.unsqueeze(dim=2))
        #                 #print(bij.shape)
        #                 #print(bij)
        #                 bi.append(bij.squeeze())
        #             bi = torch.stack(bi)

        #             bi = bi.T
        #             #print(bi.shape)
        #             #print(bi)
        #             b.append(bi)



        #         b = torch.stack(b)
        #         b = b.transpose(0,1)

        #         print(b)
        #         print(b.shape)


        #print(0/0)

        ####################################################






        #c = F.softmax(b,dim=1)

        #print(c)
        #print(c.shape)
        #print(0/0)
        ############################################################

        #print

        #c = 32*64*17

        #u_j_i = 32*768


        #cij = 32*1


        #         kk = []
        #         for i in range(17):
        #             si = 0
        #             for j in range(64):
        #                 #print(inputs_embeds[:,j,:].shape)
        #                 #print(self.W_ij)
        #                 #print(self.W_ij[i,j,:,:].shape)



        #                 u_j_i = torch.mm(inputs_embeds[:,j,:],self.W_ij[i,j,:,:])


        #                 #print(j,i,u_j_i)
        #                 #print(0/0)

        #                 cij = c[:,j,i].unsqueeze(dim=1)
        #                 #print('cij ',cij)
        #                 #print(cij.shape)
        #                 #print(u_j_i)
        #                 #print(u_j_i.shape)
        #                 si+= u_j_i*cij

        #                 #print(u_j_i*cij)
        #             #print(si)
        #             #print(0/0)

        #             norm = torch.norm(si, dim=1)

        #             #print(norm)
        #             #print(norm.shape)

        #             vi = ((norm**2)/(1+norm**2)).unsqueeze(dim=1) * (si/norm.unsqueeze(dim=1))

        #             #print(vi)
        #             kk.append(torch.norm(vi,dim=1))
        #             #kk.append(vi)
        #             #print(kk)


        #         kk = torch.stack(kk)

        #         print('kk', kk)
        #         print('kk1', kk1)
        #         print(kk.shape)
        #         print(kk1.shape)








        # Create a tensor of ones with the same number of columns as the original tensor
        ones_column_vam = torch.ones((visual_attention_mask.size(0), 1), dtype=visual_attention_mask.dtype)

        # Concatenate the original tensor with the tensor of ones along the second dimension (columns)
        visual_attention_mask_latent = torch.cat((visual_attention_mask, ones_column_vam.to('cuda')), dim=1)

        # Create a tensor of ones with the same number of columns as the original tensor
        ones_column_vtt = torch.ones((visual_token_type_ids.size(0), 1), dtype=visual_token_type_ids.dtype)

        # Concatenate the original tensor with the tensor of ones along the second dimension (columns)
        visual_token_type_ids_latent = torch.cat((visual_token_type_ids, ones_column_vtt.to('cuda')), dim=1)

        #         print(latent.shape)

        #         print(visual_embeds.shape)
        #         print(visual_attention_mask.shape)
        #         print(visual_token_type_ids.shape)

        #         print(self.w)

        x = self.vb(input_ids = input_ids, token_type_ids = token_type_ids, attention_mask = attention_mask, visual_embeds=visual_embeds_latent, visual_attention_mask=visual_attention_mask_latent, visual_token_type_ids=visual_token_type_ids_latent)

        p = self.vb(input_ids = input_ids, token_type_ids = token_type_ids, attention_mask = attention_mask, visual_embeds=visual_embeds, visual_attention_mask=visual_attention_mask, visual_token_type_ids=visual_token_type_ids)
        p = self.grad_rev1(p.pooler_output)
        #q = self.grad_rev2(raw_latent.repeat(B,1))
        q = self.grad_rev2(raw_latent)

        logits_p = self.linear_relu_stack_hidden(p)
        logits_q = self.linear_relu_stack_latent(q)




        logits = self.linear_relu_stack(x.pooler_output)


        if_logits_p = self.linear_relu_stack(p)
        if_logits_q = self.linear_relu_stack(q)

        return logits, logits_p, logits_q, if_logits_p, if_logits_q
        #return logits, logits_p, logits_q, if_logits_p, if_logits_p


In [ ]:
#R
vb = VB()

# vb = nn.DataParallel(vb)
vb.to(device)
concept_classes = ["holocaust", "nazism", "genocide", "funny", "anti muslim", "terrorism", "violence", "politics", "racism", "international relation", "adult", "gore", "misogynistic", "immigration", "extremism", "immoral", "white supremacy", "indecency"]


concept_embedding_map = {}

all_concepts = []


for c in concept_classes:
    conc = torch.tensor(tokenizer.encode(c)[1:-1])
    e = vb.vb.embeddings.word_embeddings(conc.to('cuda'))
    x = e.detach().cpu().mean(dim=0)



    all_concepts.append(x)

    concept_embedding_map[c] = x


all_concepts = torch.stack(all_concepts)



In [ ]:
#R
all_concepts.shape

torch.Size([18, 768])

In [ ]:
#R
#'/kaggle/working/final_model_concept.pt'

vb = VB()
vb.load_state_dict(torch.load(FILE))
vb.to('cuda')
vb.eval()

VB(
  (vb): VisualBertModel(
    (embeddings): VisualBertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
      (visual_token_type_embeddings): Embedding(2, 768)
      (visual_position_embeddings): Embedding(512, 768)
      (visual_projection): Linear(in_features=2048, out_features=768, bias=True)
    )
    (encoder): VisualBertEncoder(
      (layer): ModuleList(
        (0-11): 12 x VisualBertLayer(
          (attention): VisualBertAttention(
            (self): VisualBertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1

In [ ]:
# all_concepts.shape

In [ ]:
# all_concepts[0].unsqueeze(0).shape

In [ ]:
# all_concepts[0].unsqueeze(0).repeat(32,1,1).squeeze().shape

In [ ]:
#R
# check evaluation performance separately on p and q using their classifier: should be less
pred_r1, pred_r2 = [],[]
pred_both = []
act = []
for ii, batch in enumerate(eval_dataloader):
    with torch.no_grad():
        batch_inp = {k: v.to(device) for k, v in batch[0].items()}
        outputs, r1, r2,_,_ = vb(all_concepts.to('cuda'),**batch_inp)
        act.append(batch[1].detach().cpu().numpy()[0])

        pred_r1.append(np.argmax(r1.detach().cpu().numpy()[0]))
        pred_r2.append(np.argmax(r2.detach().cpu().numpy()[0]))
        pred_both.append(np.argmax(outputs.detach().cpu().numpy()[0]))

print('*')
print(f1_score(act,pred_r1,average='macro'))
print(f1_score(act,pred_r1,average='weighted'))
print(accuracy_score(act,pred_r1))
print('*'*10)
print(f1_score(act,pred_r2,average='macro'))
print(f1_score(act,pred_r2,average='weighted'))
print(accuracy_score(act,pred_r2))
print('*'*10)
print(f1_score(act,pred_both,average='macro'))
print(f1_score(act,pred_both,average='weighted'))
print(accuracy_score(act,pred_both))

*
0.40647482014388486
0.5567473294091998
0.6848484848484848
**********
0.40647482014388486
0.5567473294091998
0.6848484848484848
**********
0.7036565354754966
0.7469814601650707
0.75


In [ ]:
# check evaluation performance separately on p and q using parent classifier: should be also less
pred_r1, pred_r2 = [],[]
pred_both = []
act = []
for ii, batch in enumerate(eval_dataloader):
    with torch.no_grad():
        batch_inp = {k: v.to(device) for k, v in batch[0].items()}
        outputs, _,_, r1, r2 = vb(all_concepts.to('cuda'),**batch_inp)
        act.append(batch[1].detach().cpu().numpy()[0])

        pred_r1.append(np.argmax(r1.detach().cpu().numpy()[0]))
        pred_r2.append(np.argmax(r2.detach().cpu().numpy()[0]))
        pred_both.append(np.argmax(outputs.detach().cpu().numpy()[0]))

print('*'*10)
print(f1_score(act,pred_r1,average='macro'))
print(f1_score(act,pred_r1,average='weighted'))
print(accuracy_score(act,pred_r1))
print('*'*10)
print(f1_score(act,pred_r2,average='macro'))
print(f1_score(act,pred_r2,average='weighted'))
print(accuracy_score(act,pred_r2))
print('*'*10)
print(f1_score(act,pred_both,average='macro'))
print(f1_score(act,pred_both,average='weighted'))
print(accuracy_score(act,pred_both))

**********
0.23963133640552994
0.1510403574919704
0.3151515151515151
**********
0.23963133640552994
0.1510403574919704
0.3151515151515151
**********
0.7036565354754966
0.7469814601650707
0.75


In [ ]:
#torch.save(vb.state_dict(), '/kaggle/working/final_model_wo_adversarial.pt')

In [ ]:
#R
from typing import Dict

import numpy as np
import scipy



# import svm_classifier

REGRESSION = False
from typing import List
from tqdm import tqdm
import random

def get_weights(model) -> np.ndarray:
        """
        :return: final weights of the model, as np array
        """

        w = model.coef_
        if len(w.shape) == 1:
                w = np.expand_dims(w, 0)

        return w


def train_network(model, X_train: np.ndarray, Y_train: np.ndarray, X_dev: np.ndarray, Y_dev: np.ndarray) -> float:

        """
        :param X_train:
        :param Y_train:
        :param X_dev:
        :param Y_dev:
        :return: accuracy score on the dev set / Person's R in the case of regression
        """

        model.fit(X_train, Y_train)
        score = model.score(X_dev, Y_dev)
        return score


from typing import Dict

import numpy as np
import scipy


# import svm_classifier

REGRESSION = False
from typing import List
from tqdm import tqdm
import random


def get_nullspace_projection(W: np.ndarray) -> np.ndarray:
    """
    :param W: the matrix over its nullspace to project
    :return: the projection matrix
    """
    nullspace_basis = scipy.linalg.null_space(W)  # orthogonal basis
    nullspace_basis = nullspace_basis * np.sign(nullspace_basis[0][0])  # handle sign ambiguity
    projection_matrix = nullspace_basis.dot(nullspace_basis.T)

    return projection_matrix

from sklearn import svm

In [ ]:

#R
num_concepts = 18
BS = 8
import torch
from torch import nn
from pytorch_revgrad import RevGrad
class VB(nn.Module):
    def __init__(self):
        super(VB, self).__init__()
        self.vb = VisualBertModel.from_pretrained("uclanlp/visualbert-vqa-coco-pre")

        self.linear_relu_stack = nn.Sequential(
            nn.Dropout(0.1),
            nn.Linear(768, 512),
            nn.ReLU(),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Linear(128, 2),
        )


        self.linear_relu_stack_latent = nn.Sequential(
            nn.Dropout(0.1),
            nn.Linear(768, 512),
            nn.ReLU(),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Linear(128, 2),
        )


        self.linear_relu_stack_hidden = nn.Sequential(
            nn.Dropout(0.1),
            nn.Linear(768, 512),
            nn.ReLU(),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Linear(128, 2),
        )

        #self.w = torch.nn.Parameter(torch.ones(11,1)/11)
        self.w = torch.nn.Parameter(torch.ones(num_concepts,1))

        self.project_latent = nn.Linear(768,2048)

        #self.W_ij = [[nn.Linear(768,768) for i in range(64)] for j in range(17)]

        self.W_ij = torch.nn.Parameter(torch.randn(18,64,28,28))

        self.proj_conc = nn.Linear(768,28)
        self.proj_embed = nn.Linear(768,28)



        self.grad_rev1 = RevGrad()
        self.grad_rev2 = RevGrad()

        self.w1 = torch.nn.Parameter(torch.tensor(0.2))


    def forward(self,
        all_concepts_,
        input_ids = None,
        attention_mask = None,
        token_type_ids = None,
        position_ids = None,
        head_mask = None,
        inputs_embeds = None,
        visual_embeds = None,
        visual_attention_mask = None,
        visual_token_type_ids = None,
        image_text_alignment = None,
        output_attentions = None,
        output_hidden_states = None,
        return_dict = None):

        #         print(all_concepts.shape)

        B = visual_embeds.size(0)

        #         print(B)




        ##############################################

        all_concepts = self.proj_conc(all_concepts_)



        with torch.no_grad():
            inputs_embeds = self.vb.embeddings.word_embeddings(input_ids)

        #print(0/0)

        inputs_embeds = self.proj_embed(inputs_embeds)


        #####################################################





        #b1 = torch.bmm(inputs_embeds,all_concepts.repeat(B,1,1).transpose(1,2))

        #print(inputs_embeds.shape)
        #print(all_concepts.repeat(B,1,1).transpose(1,2).shape)

        #b,64,28
        #b,28,17


        b = torch.einsum('blx,bxn->bln',inputs_embeds,all_concepts.repeat(B,1,1).transpose(1,2))

        #         print(b1)

        #         print('********')

        #         print(b2)

        #         print(torch.equal(b1,b2))



        b = b.masked_fill(~attention_mask.unsqueeze(dim=2).bool(), -np.inf)
        c = F.softmax(b,dim=1)



        #inter = torch.einsum('bcsn,csnn->bcsn',inputs_embeds.unsqueeze(1).repeat(1,17,1,1), self.W_ij)

        #print(inter[:,0,0,:])

        einsum_expression = 'bln,clnx->bclx'
        inter = torch.einsum(einsum_expression, inputs_embeds, self.W_ij)

        k = torch.einsum('bcsn,bsc->bcn',inter,c)
        norm = torch.norm(k,dim=2)



        kk1 = ((norm**2)/(1+norm**2)).unsqueeze(dim=2) * (k/norm.unsqueeze(dim=2))

        w_dynamic = kk1.norm(dim=2).unsqueeze(2)



        raw_latent = (1-self.w1) * all_concepts_.unsqueeze(0).repeat(B,1,1) * self.w.unsqueeze(dim=0).repeat(B,1,1) + all_concepts_.unsqueeze(0).repeat(B,1,1) * w_dynamic * self.w1


        #raw_latent = 0 * all_concepts_.unsqueeze(0).repeat(B,1,1) * self.w.unsqueeze(dim=0).repeat(B,1,1) + all_concepts_.unsqueeze(0).repeat(B,1,1) * w_dynamic *1


        raw_latent = torch.mean(raw_latent, dim=1)

        #raw_latent = torch.mean(all_concepts*self.w,dim=0)
        #raw_latent = raw_latent.unsqueeze(dim=0)

        latent = self.project_latent(raw_latent)



        latent = latent.unsqueeze(dim=1)




        visual_embeds_latent = torch.cat((visual_embeds,latent),dim=1)


        ################################################################


        #print(b)

        #######################################################
        #         print(b.shape)

        #         b = []
        #         for i in range(64):
        #             bi = []
        #             for j in range(17):
        #                 #print(inputs_embeds[:,i,:].shape)
        #                 ci = all_concepts[j].unsqueeze(0).repeat(32,1,1).squeeze()
        #                 uj = inputs_embeds[:,i,:]
        #                 #print(ci.shape)
        #                 bij = torch.bmm(uj.unsqueeze(dim=1), ci.unsqueeze(dim=2))
        #                 #print(bij.shape)
        #                 #print(bij)
        #                 bi.append(bij.squeeze())
        #             bi = torch.stack(bi)

        #             bi = bi.T
        #             #print(bi.shape)
        #             #print(bi)
        #             b.append(bi)



        #         b = torch.stack(b)
        #         b = b.transpose(0,1)

        #         print(b)
        #         print(b.shape)


        #print(0/0)

        ####################################################






        #c = F.softmax(b,dim=1)

        #print(c)
        #print(c.shape)
        #print(0/0)
        ############################################################

        #print

        #c = 32*64*17

        #u_j_i = 32*768


        #cij = 32*1


        #         kk = []
        #         for i in range(17):
        #             si = 0
        #             for j in range(64):
        #                 #print(inputs_embeds[:,j,:].shape)
        #                 #print(self.W_ij)
        #                 #print(self.W_ij[i,j,:,:].shape)



        #                 u_j_i = torch.mm(inputs_embeds[:,j,:],self.W_ij[i,j,:,:])


        #                 #print(j,i,u_j_i)
        #                 #print(0/0)

        #                 cij = c[:,j,i].unsqueeze(dim=1)
        #                 #print('cij ',cij)
        #                 #print(cij.shape)
        #                 #print(u_j_i)
        #                 #print(u_j_i.shape)
        #                 si+= u_j_i*cij

        #                 #print(u_j_i*cij)
        #             #print(si)
        #             #print(0/0)

        #             norm = torch.norm(si, dim=1)

        #             #print(norm)
        #             #print(norm.shape)

        #             vi = ((norm**2)/(1+norm**2)).unsqueeze(dim=1) * (si/norm.unsqueeze(dim=1))

        #             #print(vi)
        #             kk.append(torch.norm(vi,dim=1))
        #             #kk.append(vi)
        #             #print(kk)


        #         kk = torch.stack(kk)

        #         print('kk', kk)
        #         print('kk1', kk1)
        #         print(kk.shape)
        #         print(kk1.shape)








        # Create a tensor of ones with the same number of columns as the original tensor
        ones_column_vam = torch.ones((visual_attention_mask.size(0), 1), dtype=visual_attention_mask.dtype)

        # Concatenate the original tensor with the tensor of ones along the second dimension (columns)
        visual_attention_mask_latent = torch.cat((visual_attention_mask, ones_column_vam.to('cuda')), dim=1)

        # Create a tensor of ones with the same number of columns as the original tensor
        ones_column_vtt = torch.ones((visual_token_type_ids.size(0), 1), dtype=visual_token_type_ids.dtype)

        # Concatenate the original tensor with the tensor of ones along the second dimension (columns)
        visual_token_type_ids_latent = torch.cat((visual_token_type_ids, ones_column_vtt.to('cuda')), dim=1)

        #         print(latent.shape)

        #         print(visual_embeds.shape)
        #         print(visual_attention_mask.shape)
        #         print(visual_token_type_ids.shape)

        #         print(self.w)

        x = self.vb(input_ids = input_ids, token_type_ids = token_type_ids, attention_mask = attention_mask, visual_embeds=visual_embeds_latent, visual_attention_mask=visual_attention_mask_latent, visual_token_type_ids=visual_token_type_ids_latent)

        p = self.vb(input_ids = input_ids, token_type_ids = token_type_ids, attention_mask = attention_mask, visual_embeds=visual_embeds, visual_attention_mask=visual_attention_mask, visual_token_type_ids=visual_token_type_ids)
        p = self.grad_rev1(p.pooler_output)
        #q = self.grad_rev2(raw_latent.repeat(B,1))
        q = self.grad_rev2(raw_latent)

        logits_p = self.linear_relu_stack_hidden(p)
        logits_q = self.linear_relu_stack_latent(q)




        logits = self.linear_relu_stack(x.pooler_output)


        if_logits_p = self.linear_relu_stack(p)
        if_logits_q = self.linear_relu_stack(q)

        return logits, logits_p, logits_q, if_logits_p, if_logits_q
        #return logits, logits_p, logits_q, if_logits_p, if_logits_p


In [ ]:
#R
def get_inlp(all_concepts):
    print('INLP step')
    num_concepts = 18
    all_concepts_ = all_concepts.numpy()
    multiplier = np.logical_xor(1, np.eye(num_concepts)).astype(np.int32)/ (num_concepts-1)
    X = np.matmul(multiplier,all_concepts_)
    assert X.shape[0]==num_concepts
    Y = np.arange(num_concepts)

    #             x_train_cp = x_train
    #             x_test_cp = x_test
    x_train_cp = X
    x_test_cp = X
    y_train = Y
    y_test = Y
    print('******')
    for i in range(1):

        clf = svm.SVC(kernel="linear")
        scr = train_network(clf, x_train_cp, y_train, x_test_cp, y_test)
        print('initial {}'.format(scr))
        W = get_weights(clf)
        P_i = get_nullspace_projection(W)
        x_train_cp = x_train_cp.dot(P_i)
        x_test_cp = x_test_cp.dot(P_i)
        print(clf.score(x_test_cp, y_test))
        print(clf.score(x_train_cp, y_train))
        clf = svm.SVC(kernel="linear")
        train_network(clf, x_train_cp, y_train, x_test_cp, y_test)
    print('******')
    all_concepts_ = torch.tensor(all_concepts_)
    all_concepts_ = all_concepts_.mm(torch.tensor(P_i, dtype = all_concepts_.dtype))

    return all_concepts_

In [ ]:
#R
import torch
from torch import nn
from pytorch_revgrad import RevGrad
class VB_wrapper(nn.Module):
    def __init__(self, vb):
        super(VB_wrapper, self).__init__()

        self.vb = vb



    def forward(self,
        all_concepts_,
        input_embeds = None,
        visual_embeds = None,
        attention_mask = None,
        token_type_ids = None,
        position_ids = None,
        head_mask = None,
        inputs_embeds = None,

        visual_attention_mask = None,
        visual_token_type_ids = None,
        image_text_alignment = None,
        output_attentions = None,
        output_hidden_states = None,
        return_dict = None):







        #         print('ac', all_concepts_.shape)
        #         print('ii', input_embeds.shape)
        #         print('am', attention_mask)
        #         print('tti', token_type_ids)
        #         print('pi', position_ids)
        #         print('hm', head_mask)
        #         print('ie', inputs_embeds)
        #         print('ve', visual_embeds)
        #         print('vam', visual_attention_mask)
        #         print('vtti', visual_token_type_ids)
        #         print(image_text_alignment)
        #         print(output_attentions)
        #         print(output_hidden_states)
        #         print(return_dict)






        B = visual_embeds.size(0)


        all_concepts = self.vb.proj_conc(all_concepts_)



        """
        with torch.no_grad():
            inputs_embeds = self.vb.vb.embeddings.word_embeddings(input_ids.to('cuda'))
        """

        with torch.no_grad():
            inputs_embeds = self.vb.vb.embeddings.word_embeddings(input_ids.to('cuda'))

        #print(0/0)

        inputs_embeds = self.vb.proj_embed(inputs_embeds)





        b = torch.einsum('blx,bxn->bln',inputs_embeds,all_concepts.transpose(1,2))


        #print(b.shape)



        b = b.masked_fill(~attention_mask.unsqueeze(dim=2).bool(), -np.inf)
        c = F.softmax(b,dim=1)




        einsum_expression = 'bln,clnx->bclx'
        inter = torch.einsum(einsum_expression, inputs_embeds, self.vb.W_ij)

        k = torch.einsum('bcsn,bsc->bcn',inter,c)
        norm = torch.norm(k,dim=2)



        kk1 = ((norm**2)/(1+norm**2)).unsqueeze(dim=2) * (k/norm.unsqueeze(dim=2))

        w_dynamic = kk1.norm(dim=2).unsqueeze(2)

        num_reps = int(all_concepts.shape[0])
        w_static = self.vb.w.unsqueeze(0).repeat(num_reps,1,1)


        #         print('*'*10)

        #         print(self.vb.w1.shape)
        #         print(all_concepts_.shape)
        #         print(w_static.shape)
        #         print(w_dynamic.shape)



        if DYN_ROUTE:
            raw_latent = (1-self.vb.w1) * all_concepts_ * w_static + all_concepts_ * w_dynamic * self.vb.w1

        else:
            raw_latent = (1-self.vb.w1) * all_concepts_ * w_static



        #raw_latent = (1-self.vb.w1) * all_concepts_ * w_static
        raw_latent = torch.mean(raw_latent, dim=1)


        latent = self.vb.project_latent(raw_latent)



        latent = latent.unsqueeze(dim=1)




        visual_embeds_latent = torch.cat((visual_embeds,latent),dim=1)














        #         num_reps = int(all_concepts.shape[0]/17)
        #         w_static = self.vb.w.unsqueeze(0).repeat(num_reps,1,1).reshape(17*num_reps,1)
        #         raw_latent = torch.mean(all_concepts*w_static,dim=0)
        #         raw_latent = raw_latent.unsqueeze(dim=0)

        #         latent = self.vb.project_latent(raw_latent)



        #         latent = latent.repeat(B,1).unsqueeze(dim=1)


        #         visual_embeds_latent = torch.cat((visual_embeds,latent),dim=1)




        # Create a tensor of ones with the same number of columns as the original tensor
        ones_column_vam = torch.ones((visual_attention_mask.size(0), 1), dtype=visual_attention_mask.dtype)

        # Concatenate the original tensor with the tensor of ones along the second dimension (columns)
        visual_attention_mask_latent = torch.cat((visual_attention_mask, ones_column_vam.to('cuda')), dim=1)

        # Create a tensor of ones with the same number of columns as the original tensor
        ones_column_vtt = torch.ones((visual_token_type_ids.size(0), 1), dtype=visual_token_type_ids.dtype)

        # Concatenate the original tensor with the tensor of ones along the second dimension (columns)
        visual_token_type_ids_latent = torch.cat((visual_token_type_ids, ones_column_vtt.to('cuda')), dim=1)

        #         print(latent.shape)

        #         print(visual_embeds.shape)
        #         print(visual_attention_mask.shape)
        #         print(visual_token_type_ids.shape)

        #         print(self.w)

        x = self.vb.vb(inputs_embeds = input_embeds, token_type_ids = token_type_ids, attention_mask = attention_mask, visual_embeds=visual_embeds_latent, visual_attention_mask=visual_attention_mask_latent, visual_token_type_ids=visual_token_type_ids_latent)






        logits = self.vb.linear_relu_stack(x.pooler_output)
        #print('softmax logit', torch.softmax(logits, dim=1))
        #print(0/0)

        #logits = torch.softmax(logits, dim=1)[:, 1].unsqueeze(1)

        #print('previous',torch.softmax(logits, dim=1)[:, 1].unsqueeze(1) )

        logits = torch.softmax(logits, dim=1).max(dim=1).values.unsqueeze(1)

        #print('logits', logits)

        #print(0/0)

        #print(logits)
        #print(logits.shape)



        return logits



In [ ]:
#R
#'/kaggle/working/final_model_concept.pt'

vb = VB()

vb.load_state_dict(torch.load(FILE))
vb.to('cuda')
vb.eval()

VB(
  (vb): VisualBertModel(
    (embeddings): VisualBertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
      (visual_token_type_embeddings): Embedding(2, 768)
      (visual_position_embeddings): Embedding(512, 768)
      (visual_projection): Linear(in_features=2048, out_features=768, bias=True)
    )
    (encoder): VisualBertEncoder(
      (layer): ModuleList(
        (0-11): 12 x VisualBertLayer(
          (attention): VisualBertAttention(
            (self): VisualBertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1

In [ ]:
#R
vb_c = VB()
vb_c.to(device)
# concept_classes = ["jew", "holocaust", "hitler", "genocide", "funny", "muslim", "terrorism", "violence", "politics", "naive", "international relation"]


# concept_embedding_map = {}

# all_concepts = []


# for c in concept_classes:
#     conc = torch.tensor(tokenizer.encode(c)[1:-1])
#     e = vb_c.vb.embeddings.word_embeddings(conc.to('cuda'))
#     x = e.detach().cpu().mean(dim=0)

#     all_concepts.append(x)

#     concept_embedding_map[c] = x


# all_concepts = torch.stack(all_concepts)

# concept_classes = ["jew", "holocaust", "hitler", "genocide", "funny", "muslim", "terrorism", "violence", "politics", "racism", "international relation", "sex", "gore", "violence", "immigration", "extremism", "immoral"]

concept_classes = ["holocaust", "nazism", "genocide", "funny", "anti muslim", "terrorism", "violence", "politics", "racism", "international relation", "adult", "gore", "misogynistic", "immigration", "extremism", "immoral", "white supremacy", "indecency"]


concept_embedding_map = {}

all_concepts = []


for c in concept_classes:
    conc = torch.tensor(tokenizer.encode(c)[1:-1])
    e = vb_c.vb.embeddings.word_embeddings(conc.to('cuda'))
    x = e.detach().cpu().mean(dim=0)


    all_concepts.append(x)

    concept_embedding_map[c] = x


all_concepts = torch.stack(all_concepts)



In [ ]:
#R
def get_inlp(all_concepts):

    num_concepts = 18
    all_concepts_ = all_concepts.cpu().numpy()
    multiplier = np.logical_xor(1, np.eye(num_concepts)).astype(np.int32)/ (num_concepts-1)
    X = np.matmul(multiplier,all_concepts_)
    assert X.shape[0]==num_concepts
    Y = np.arange(num_concepts)

    #             x_train_cp = x_train
    #             x_test_cp = x_test
    x_train_cp = X
    x_test_cp = X
    y_train = Y
    y_test = Y

    for i in range(1):

        clf = svm.SVC(kernel="linear")
        scr = train_network(clf, x_train_cp, y_train, x_test_cp, y_test)

        W = get_weights(clf)
        P_i = get_nullspace_projection(W)
        x_train_cp = x_train_cp.dot(P_i)
        x_test_cp = x_test_cp.dot(P_i)

        clf = svm.SVC(kernel="linear")
        train_network(clf, x_train_cp, y_train, x_test_cp, y_test)

    all_concepts_ = torch.tensor(all_concepts_)
    all_concepts_ = all_concepts_.mm(torch.tensor(P_i, dtype = all_concepts_.dtype))

    return all_concepts_

In [ ]:
#R
#torch.save(vb.state_dict(), '/kaggle/working/final_model_concept_inlp.pt')
# check evaluation performance separately on p and q using their classifier: should be less
pred_r1, pred_r2 = [],[]
pred_both = []
act = []
if DECONFOUND:
    all_concepts_ = get_inlp(all_concepts).to('cuda')
else:
    all_concepts_ = all_concepts.to('cuda')

for ii, batch in enumerate(eval_dataloader):
    with torch.no_grad():
        batch_inp = {k: v.to(device) for k, v in batch[0].items()}
        outputs, r1, r2,_,_ = vb(all_concepts_.to('cuda'),**batch_inp)
        act.append(batch[1].detach().cpu().numpy()[0])

        pred_r1.append(np.argmax(r1.detach().cpu().numpy()[0]))
        pred_r2.append(np.argmax(r2.detach().cpu().numpy()[0]))
        pred_both.append(np.argmax(outputs.detach().cpu().numpy()[0]))

print('*')
print(f1_score(act,pred_r1,average='macro'))
print(f1_score(act,pred_r1,average='weighted'))
print(accuracy_score(act,pred_r1))
print('*'*10)
print(f1_score(act,pred_r2,average='macro'))
print(f1_score(act,pred_r2,average='weighted'))
print(accuracy_score(act,pred_r2))
print('*'*10)
print(f1_score(act,pred_both,average='macro'))
print(f1_score(act,pred_both,average='weighted'))
print(accuracy_score(act,pred_both))

*
0.40647482014388486
0.5567473294091998
0.6848484848484848
**********
0.40647482014388486
0.5567473294091998
0.6848484848484848
**********
0.7036565354754966
0.7469814601650707
0.75


In [ ]:
print('h')

h


In [ ]:
#R
def model_pred(input_ids, attention_mask, token_type_ids, visual_embeds, visual_attention_mask, visual_token_type_ids, all_concepts, concept_no=None):



    B = visual_embeds.size(0)


    #######################################################
    input_ids = input_ids.to('cuda')
    attention_mask = attention_mask.to('cuda')
    token_type_ids = token_type_ids.to('cuda')
    visual_embeds = visual_embeds.to('cuda')
    visual_attention_mask = visual_attention_mask.to('cuda')
    visual_token_type_ids = visual_token_type_ids.to('cuda')

    #####################################################################

    ##############################################

    all_concepts = all_concepts.to('cuda')

    # deconfounding step by passing the concepts through INLP (1)
    if DECONFOUND:
        all_concepts_ = get_inlp(all_concepts).to('cuda')
    else:
        all_concepts_ = all_concepts.to('cuda')

    # this is without deconfounding while testing (2)
    # either use (1) or (2) or mean of (1) and (2)

    # all_concepts_ = all_concepts.to('cuda')

    #all_concepts_ = (get_inlp(all_concepts).to('cuda') + all_concepts.to('cuda')) / 2

    all_concepts = vb.proj_conc(all_concepts_)



    with torch.no_grad():
        inputs_embeds = vb.vb.embeddings.word_embeddings(input_ids.to('cuda'))

    #print(0/0)

    inputs_embeds = vb.proj_embed(inputs_embeds)


    #####################################################





    #B, 64, 28, #B, 28, 17

    b = torch.einsum('blx,bxn->bln',inputs_embeds,all_concepts.repeat(B,1,1).transpose(1,2))



    b = b.masked_fill(~attention_mask.unsqueeze(dim=2).bool(), -np.inf)
    c = F.softmax(b,dim=1)




    einsum_expression = 'bln,clnx->bclx'
    inter = torch.einsum(einsum_expression, inputs_embeds, vb.W_ij)

    k = torch.einsum('bcsn,bsc->bcn',inter,c)
    norm = torch.norm(k,dim=2)



    kk1 = ((norm**2)/(1+norm**2)).unsqueeze(dim=2) * (k/norm.unsqueeze(dim=2))

    w_dynamic = kk1.norm(dim=2).unsqueeze(2)

    #     print(w_dynamic.shape)
    #     print(vb.w.shape)


    #     print(w_dynamic)
    #     print(vb.w)
    #     print('*'*10)


    #print(concept_no)
    if concept_no==None:
        k_static = vb.w.data.clone()
        k_dynamic = w_dynamic.data.clone()
    else:
        k_static = vb.w.data.clone()
        k_dynamic = w_dynamic.data.clone()
        k_static[concept_no][0] = 0
        k_dynamic[0][concept_no][0] = 0

    #     print(k_dynamic)
    #     print(k_static)
    #     print('*'*10)

    #     print(0/0)

    #either use the following line or raw_latent = (1-vb.w1) * all_concepts_.unsqueeze(0).repeat(B,1,1) * k_static.unsqueeze(dim=0).repeat(B,1,1)

    if DYN_ROUTE:
        raw_latent = (1-vb.w1) * all_concepts_.unsqueeze(0).repeat(B,1,1) * k_static.unsqueeze(dim=0).repeat(B,1,1) + all_concepts_.unsqueeze(0).repeat(B,1,1) * k_dynamic * vb.w1
    else:
        raw_latent = (1-vb.w1) * all_concepts_.unsqueeze(0).repeat(B,1,1) * k_static.unsqueeze(dim=0).repeat(B,1,1)

    #raw_latent = 0.0 * all_concepts_.unsqueeze(0).repeat(B,1,1) * k_static.unsqueeze(dim=0).repeat(B,1,1) + all_concepts_.unsqueeze(0).repeat(B,1,1) * k_dynamic * 1.0

    #raw_latent = (1-vb.w1) * all_concepts_.unsqueeze(0).repeat(B,1,1) * k_static.unsqueeze(dim=0).repeat(B,1,1)

    raw_latent = torch.mean(raw_latent, dim=1)



    latent = vb.project_latent(raw_latent)



    latent = latent.unsqueeze(dim=1)




    visual_embeds_latent = torch.cat((visual_embeds,latent),dim=1)


    ################################################################




    # Create a tensor of ones with the same number of columns as the original tensor
    ones_column_vam = torch.ones((visual_attention_mask.size(0), 1), dtype=visual_attention_mask.dtype)

    # Concatenate the original tensor with the tensor of ones along the second dimension (columns)
    visual_attention_mask_latent = torch.cat((visual_attention_mask.to('cuda'), ones_column_vam.to('cuda')), dim=1)

    # Create a tensor of ones with the same number of columns as the original tensor
    ones_column_vtt = torch.ones((visual_token_type_ids.size(0), 1), dtype=visual_token_type_ids.dtype)

    # Concatenate the original tensor with the tensor of ones along the second dimension (columns)
    visual_token_type_ids_latent = torch.cat((visual_token_type_ids.to('cuda'), ones_column_vtt.to('cuda')), dim=1)



    x = vb.vb(input_ids = input_ids.to('cuda'), token_type_ids = token_type_ids.to('cuda'), attention_mask = attention_mask.to('cuda'), visual_embeds=visual_embeds_latent.to('cuda'), visual_attention_mask=visual_attention_mask_latent.to('cuda'), visual_token_type_ids=visual_token_type_ids_latent.to('cuda'))

    p = vb.vb(input_ids = input_ids.to('cuda'), token_type_ids = token_type_ids.to('cuda'), attention_mask = attention_mask.to('cuda'), visual_embeds=visual_embeds.to('cuda'), visual_attention_mask=visual_attention_mask.to('cuda'), visual_token_type_ids=visual_token_type_ids.to('cuda'))
    p = vb.grad_rev1(p.pooler_output)
    q = vb.grad_rev2(raw_latent.repeat(B,1))

    logits_p = vb.linear_relu_stack_hidden(p)
    logits_q = vb.linear_relu_stack_latent(q.to('cuda'))




    logits = vb.linear_relu_stack(x.pooler_output)


    if_logits_p = vb.linear_relu_stack(p)
    if_logits_q = vb.linear_relu_stack(q.to('cuda'))

    return torch.softmax(logits,dim=-1), x.pooler_output

In [ ]:
from PIL import Image

In [ ]:
for i in range(50):
    if train_set[i]['label']==1:
        print(i, train_set[i])
        Image.open('/kaggle/input/facebook-hateful-meme-dataset/data/'+train_set[i]['img'])
        inputs = {}
        inputs = tokenizer(train_set[i]['text'], padding="max_length", truncation=True, max_length=64, return_tensors='pt')


        inputs['input_ids'] = inputs['input_ids']
        inputs['token_type_ids'] = inputs['token_type_ids']
        inputs['attention_mask'] = inputs['attention_mask']

        input_ids = inputs['input_ids']
        token_type_ids = inputs['token_type_ids']
        attention_mask = inputs['attention_mask']

        visual_embeds,visual_token_type_ids,visual_attention_mask = get_visual_embedding('/kaggle/input/facebook-hateful-meme-dataset/data/'+train_set[i]['img'])
        # visual_embeds,visual_token_type_ids,visual_attention_mask = visual_feats[train_set[874]['img']]



        inputs.update({
          "visual_embeds":  visual_embeds,
          "visual_token_type_ids": visual_token_type_ids,
          "visual_attention_mask": visual_attention_mask
        })


        class_map = {0:'non offensive', 1: 'offensive'}
        #print('original model prediction {}'.format(model_pred(input_ids,attention_mask,token_type_ids,visual_embeds,visual_attention_mask,visual_token_type_ids)))
        orig,_ = model_pred(input_ids,attention_mask,token_type_ids,visual_embeds,visual_attention_mask,visual_token_type_ids,all_concepts)
        orig = orig[0]
        print(orig)
        print('predicted class {}'.format(class_map[np.argmax(orig.detach().cpu().numpy())]))
        pred_class = np.argmax(orig.detach().cpu().numpy())
        orig = orig[pred_class]

        k = []
        for i in range(18):
            #print('Removing concept: {}'.format(concept_classes[i]))

            after_intervention,_ = model_pred(input_ids,attention_mask,token_type_ids,visual_embeds,visual_attention_mask,visual_token_type_ids,all_concepts,concept_no=i)
            #print('hello')
            after_intervention = after_intervention[0][pred_class]
            #print(after_intervention, orig)
            #print(0/0)
            k.append(abs(after_intervention-orig).item())
            #print('model prediction after removal {}'.format(model_pred(input_ids,attention_mask,token_type_ids,visual_embeds,visual_attention_mask,visual_token_type_ids,concept_no=i)))
        k = np.asarray(k)
        print(k)
        k = list(map(lambda x: ((x-np.mean(k))/np.mean(k))*100, k))

        conc_scr = []
        for i in range(17):
            cc = concept_classes[i]
            print('relative ICaCE scores for concept {} is {}%'.format(cc,k[i]))
            conc_scr.append((cc,k[i]))
        conc_scr.sort(key=lambda tup: tup[1], reverse=True)

        for i in conc_scr:
            if i[1]> 0:
                print(i[0], end = " ")
        print("\n")



10 {'text': 'jew mad? get fuhrerious!', 'img': 'img/79351.png', 'label': 1}


/opt/conda/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /usr/local/src/pytorch/aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


tensor([0.2246, 0.7754], device='cuda:0', grad_fn=<SelectBackward0>)
predicted class offensive
[0.00029802 0.00029796 0.00029647 0.0002957  0.0002982  0.00029802
 0.0002982  0.0002982  0.00029701 0.00029832 0.0002982  0.00029796
 0.00029808 0.00029701 0.00029755 0.00029844 0.00029582 0.00029677]
relative ICaCE scores for concept holocaust is 0.15802710943933357%
relative ICaCE scores for concept nazism is 0.1379955040174457%
relative ICaCE scores for concept genocide is -0.362794631529751%
relative ICaCE scores for concept funny is -0.6232055020142933%
relative ICaCE scores for concept anti muslim is 0.21812192570499714%
relative ICaCE scores for concept terrorism is 0.15802710943933357%
relative ICaCE scores for concept violence is 0.21812192570499714%
relative ICaCE scores for concept politics is 0.21812192570499714%
relative ICaCE scores for concept racism is -0.18251018273276015%
relative ICaCE scores for concept international relation is 0.2581851365487729%
relative ICaCE scores f

In [ ]:
# from captum.attr import IntegratedGradients

# # input1 = torch.tensor([[1.0, 3.0], [3.0, 5.0]], requires_grad=True)
# # input2 = torch.tensor([[1.0, 4.0], [0.0, 2.0]], requires_grad=True)
# # # Initializing our toy model
# # model = ToyModel_With_Additional_Forward_Args()
# # # Applying integrated gradients on the input
# ig = IntegratedGradients(vb)
# (input1_attr, input2_attr), delta = ig.attribute((input1, input2), n_steps=100,
#                                     additional_forward_args=1, return_convergence_delta=True)
# output
# .........
# input1_attr: tensor([[0.0000, 0.0000],
#                      [0.0000, 3.3428]], grad_fn=<MulBackward0>)
# input2_attr:  tensor([[ 0.0000,  0.0000],
#                       [0.0000, -1.3371]], grad_fn=<MulBackward0>)
# approximation_error (aka delta): 0.005693793296813965

In [ ]:
#R
concept_classes

['holocaust',
 'nazism',
 'genocide',
 'funny',
 'anti muslim',
 'terrorism',
 'violence',
 'politics',
 'racism',
 'international relation',
 'adult',
 'gore',
 'misogynistic',
 'immigration',
 'extremism',
 'immoral',
 'white supremacy',
 'indecency']

In [ ]:
vb

VB(
  (vb): VisualBertModel(
    (embeddings): VisualBertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
      (visual_token_type_embeddings): Embedding(2, 768)
      (visual_position_embeddings): Embedding(512, 768)
      (visual_projection): Linear(in_features=2048, out_features=768, bias=True)
    )
    (encoder): VisualBertEncoder(
      (layer): ModuleList(
        (0-11): 12 x VisualBertLayer(
          (attention): VisualBertAttention(
            (self): VisualBertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1

In [ ]:
#R
torch.manual_seed(SEED)
random.seed(SEED)
baselines = (torch.randn(20,18,768).to('cuda'), torch.randn(20,64,768).to('cuda'), torch.randn(20,36,2048).to('cuda'))



In [ ]:
#R
from captum.attr import IntegratedGradients,Saliency,DeepLift,DeepLiftShap, GradientShap, InputXGradient, KernelShap, FeatureAblation



In [ ]:
# torch.manual_seed(42)
# print(torch.randn(2,3))

In [ ]:
#R
#vb_wrapper = VB_wrapper(vb)



In [ ]:
#R
!pip install shutup

In [ ]:
len(gc)

208

In [ ]:
def call_sim():

    import math
    # Quantitative case study (1) Faithfulness on concepts
    # Quantitative case study (2) Are they explainable?
    test_dataloader=DataLoader(t_p, shuffle=True, batch_size=1)

    pred_r1, pred_r2 = [],[]
    pred_both = []
    act = []

    multimodal_repr = []
    concept_repr = []

    concept_repr1 = []
    concept_repr2 = []
    y_p = []

    cr, rr = [],[]
    cr1,rr1 = [],[]


    ours_icace = []
    ia_icace = []

    concept_cat = {}
    for i,c in enumerate(concept_classes):
        concept_cat[c] = all_concepts[i]

    for ii, batch in tqdm(enumerate(eval_dataloader)):

        #if batch[1].detach().cpu().numpy()[0] == 1:

        ll = batch[1].detach().cpu().numpy()[0]
        y_p.append(ll)

        #print('*'*10)

        input_ids = batch[0]['input_ids'].to('cuda')
        #print(tokenizer.batch_decode(batch[0]['input_ids'], skip_special_tokens=True))
        token_type_ids = batch[0]['token_type_ids'].to('cuda')
        attention_mask = batch[0]['attention_mask'].to('cuda')
        visual_embeds = batch[0]['visual_embeds'].to('cuda')
        visual_token_type_ids = batch[0]['visual_token_type_ids'].to('cuda')
        visual_attention_mask = batch[0]['visual_attention_mask'].to('cuda')

        inputs_embeds = vb_wrapper.vb.vb.embeddings.word_embeddings(input_ids)

        #     (input1_attr, input2_attr), delta = ig.attribute(inputs = (all_concepts.to('cuda'), inputs_embeds), additional_forward_args = (attention_mask,
        #         token_type_ids,
        #         None,
        #         None,
        #         None,
        #         visual_embeds,
        #         visual_attention_mask,
        #         visual_token_type_ids), return_convergence_delta=True)

        #print(visual_embeds)

        #     (input1_attr, input2_attr, input3_attr) = ig.attribute(inputs = (all_concepts.unsqueeze(0).to('cuda'), inputs_embeds, visual_embeds), additional_forward_args = (attention_mask,
        #         token_type_ids,
        #         None,
        #         None,
        #         None,
        #         visual_attention_mask,
        #         visual_token_type_ids))
        #vb(input_ids = input_ids, token_type_ids = token_type_ids, attention_mask = attention_mask, visual_embeds=visual_embeds_latent, visual_attention_mask=visual_attention_mask_latent, visual_token_type_ids=visual_token_type_ids_latent)

        if type(ig) in [captum.attr._core.kernel_shap.KernelShap, captum.attr._core.input_x_gradient.InputXGradient, captum.attr._core.deep_lift.DeepLift, captum.attr._core.saliency.Saliency, captum.attr._core.integrated_gradients.IntegratedGradients, captum.attr._core.feature_ablation.FeatureAblation]:
                baseline = None
                #print('none')
        else:
            baseline = baselines
            #print('baseline')

        all_conc = all_concepts.to('cuda')
        if type(ig) in [captum.attr._core.input_x_gradient.InputXGradient,captum.attr._core.saliency.Saliency]:
            #print('ip_grad')
            (input1_attr, input2_attr, input3_attr) = ig.attribute(inputs = (all_conc.unsqueeze(0), inputs_embeds, visual_embeds),

                                                                additional_forward_args = (attention_mask,
            token_type_ids,
            None,
            None,
            None,
            visual_attention_mask,
            visual_token_type_ids))

        else:
            #print('others')

            (input1_attr, input2_attr, input3_attr) = ig.attribute(inputs = (all_conc.unsqueeze(0), inputs_embeds, visual_embeds),
                                                                  baselines = baseline,
                                                                    additional_forward_args = (attention_mask,
                token_type_ids,
                None,
                None,
                None,
                visual_attention_mask,
                visual_token_type_ids))


        #print(input1_attr, input2_attr)
        #print(input1_attr.shape, input2_attr.shape)
        k1=input1_attr.sum(dim=-1).squeeze(0).detach().cpu().numpy()

        conc_scr = []
        attribution_score = {}
        for i in range(18):
            cc = concept_classes[i]
            #print('relative attribution scores for concept {} is {}'.format(cc,k[i]))
            conc_scr.append((cc,k1[i]))
            attribution_score[cc] = k1[i]
            #conc_scr[cc] = k1[i]
        conc_scr.sort(key=lambda tup: tup[1], reverse=True)

        d = {}

        for cnt,i in enumerate(conc_scr):
            d[i[0]] = cnt

        #print(d)


        orig,pooled_op = model_pred(input_ids,attention_mask,token_type_ids,visual_embeds,visual_attention_mask,visual_token_type_ids,all_concepts)
        orig = orig[0]
        pooled_op = pooled_op.detach().cpu()
        multimodal_repr.append(pooled_op)
        #print('predicted class {}'.format(class_map[np.argmax(orig.detach().cpu().numpy())]))
        #print('actual class {}'.format(ll))
        #print(0/0)
        #orig = orig[0]
        k = []
        pred_class = np.argmax(orig.detach().cpu().numpy())
        causal_score = {}
        #print(pred_class)
        for i in range(18):
            #print('Removing concept: {}'.format(concept_classes[i]))
            cc = concept_classes[i]
            after_intervention,_ = model_pred(input_ids,attention_mask,token_type_ids,visual_embeds,visual_attention_mask,visual_token_type_ids,all_concepts,concept_no=i)
            after_intervention = after_intervention[0][pred_class]
            #print('hello')
            #print(orig[pred_class], after_intervention)
            #print(0/0)
            k.append((cc,abs(after_intervention-orig[pred_class]).item()))
            causal_score[cc] = abs(after_intervention-orig[pred_class]).item()
            #print('model prediction after removal {}'.format(model_pred(input_ids,attention_mask,token_type_ids,visual_embeds,visual_attention_mask,visual_token_type_ids,concept_no=i)))
        #k = np.asarray(k)
        #k = np.asarray(k)
        #k = list(map(lambda x: ((x-np.mean(k))/np.mean(k))*100, k))
        k.sort(key=lambda tup: tup[1], reverse=True)
        #print(k)

        #print(list(map(lambda x: x[0], list(k))))

        gamma = 0.9
        temp = []
        for cnt,i in enumerate(list(map(lambda x: x[0], list(k)))):
            temp.append((gamma**cnt) * concept_cat[i])
            #temp.append((math.e**(-cnt)) * concept_cat[i])
            #temp.append((1.1**(-cnt)) * concept_cat[i])

        temp = torch.stack(temp).mean(dim=0).unsqueeze(dim=0)
        #print(temp)
        #print(temp.shape)

        concept_repr.append(temp)



        temp = []
        for cnt,i in enumerate(list(map(lambda x: x[0], list(conc_scr)))):
            temp.append((gamma**cnt) * concept_cat[i])
            #temp.append((math.e**(-cnt)) * concept_cat[i])
            #temp.append((1.1**(-cnt)) * concept_cat[i])

        temp = torch.stack(temp).mean(dim=0).unsqueeze(dim=0)
        #print(temp)
        #print(temp.shape)

        concept_repr1.append(temp)


        #print(list(map(lambda x: x[0], list(conc_scr))))
        #print(0/0)

        #considering the k to be the causally sorted rank, we estimate ranks of

        ip_attrib_causal = []



        for i in range(18):
            cc = concept_classes[i]
            ip_attrib_causal.append((cc,((2*attribution_score[cc]*causal_score[cc]) / (attribution_score[cc]+causal_score[cc]))))





        ip_attrib_causal.sort(key=lambda tup: tup[1], reverse=True)



        ra_icace_ours = []
        for cnt,i in enumerate(list(map(lambda x: x[0], list(k)))):
            ra_icace_ours.append((gamma**cnt) * causal_score[i])
            #ra_icace_ours.append((math.e**(-cnt)) * causal_score[i])
            #ra_icace_ours.append((1.1**(-cnt)) * causal_score[i])

        ra_icace_ia = []
        for cnt,i in enumerate(list(map(lambda x: x[0], list(conc_scr)))):
            ra_icace_ia.append((gamma**cnt) * causal_score[i])
            #ra_icace_ia.append((math.e**(-cnt)) * causal_score[i])
            #ra_icace_ia.append((1.1**(-cnt)) * causal_score[i])



        ours_icace.append(np.mean(ra_icace_ours))
        ia_icace.append(np.mean(ra_icace_ia))




        temp = []
        for cnt,i in enumerate(list(map(lambda x: x[0], list(ip_attrib_causal)))):
            temp.append((gamma**cnt) * concept_cat[i])
            #temp.append((math.e**(-cnt)) * concept_cat[i])
            #temp.append((1.1**(-cnt)) * concept_cat[i])

        temp = torch.stack(temp).mean(dim=0).unsqueeze(dim=0)

        concept_repr2.append(temp)



        d1 = {}

        for cnt,i in enumerate(ip_attrib_causal):
            d1[i[0]] = cnt

        r1,r2,r3 = [],[],[]
        for cnt,i in enumerate(list(map(lambda x: x[0], list(k)))):
            r1.append(cnt)
            r2.append(d[i])
            r3.append(d1[i])

        #print(r1,r2)
        corr, _ = stats.kendalltau(r1, r2)
        res,_ = stats.spearmanr(r1, r2)

        corr1, _ = stats.kendalltau(r1, r3)
        res1,_ = stats.spearmanr(r1, r3)

        #print('kendall tau', corr)
        cr.append(corr)
        #print('spearmans rho', res)
        rr.append(res)

        cr1.append(corr1)
        rr1.append(res1)







    multimodal_repr = torch.stack(multimodal_repr).squeeze(1)

    #causal
    concept_repr = torch.stack(concept_repr).squeeze(1)


    #attribution
    concept_repr1 = torch.stack(concept_repr1).squeeze(1)

    #HM
    concept_repr2 = torch.stack(concept_repr2).squeeze(1)


    return np.mean(cr), np.mean(rr), multimodal_repr, concept_repr, concept_repr1, y_p










In [ ]:
!ls /kaggle/working

25461.png	   cr1.pt			  mean_comparison_plot.png
28076.png	   cr2.pt			  mr.pt
94162.png	   final_model_concept.pt	  rr.npy
96704.png	   final_model_concept_inlp.pt	  rr1.npy
causal_vb.pt	   final_model_wo_adversarial.pt  transformers
causal_vb_inlp.pt  foo1.png			  vs_tensors.pt
cr.npy		   full_annotation.pkl		  y_p.pt
cr.pt		   image1.jpg
cr1.npy		   image2.jpg


In [ ]:
print(DECONFOUND, DYN_ROUTE)

True True


In [ ]:

#clf = svm.SVC(kernel='poly', C=2, random_state=42)

def get_sim(multimodal_repr, concept_repr1, y_p):

    X = np.concatenate([multimodal_repr.numpy(),concept_repr1.numpy()],axis=-1) # inp w/ clip representation w/ exp -> w/ both
    #X = (multimodal_repr.numpy()*concept_repr.numpy())
    X_ = multimodal_repr.numpy() # inp w/ clip representation w/o exp -> w/ only input
    X__ = concept_repr1.numpy() # inp w/o clip representation w/ exp -> w/ only exp

    print(X.shape)
    print(X_.shape)
    print(X__.shape)
    print(y_p)
    #print(0/0)



    kf = KFold(n_splits=5)
    y = np.array(y_p)
    vals = []
    vals_lus = []
    k1,k2,k3 = [],[],[]
    C,S = [],[]
    for train, test in kf.split(X):

        X_train, X_test, y_train, y_test = X[train], X[test], y[train], y[test]
        X_train_, X_test_, y_train, y_test = X_[train], X_[test], y[train], y[test]
        X_train__, X_test__, y_train, y_test = X__[train], X__[test], y[train], y[test]


        clf1 = make_pipeline(StandardScaler(),svm.SVC(probability=True, kernel='rbf', C=2, random_state=42))
        clf1.fit(X_train, y_train)
        y_pred = clf1.predict(X_test)

        clf2 = make_pipeline(StandardScaler(),svm.SVC(probability=True,kernel='rbf', C=2, random_state=42))
        clf2.fit(X_train_, y_train)
        y_pred_ = clf2.predict(X_test_)

        clf3 = make_pipeline(StandardScaler(), svm.SVC(probability=True,kernel='rbf', C=2, random_state=42))
        clf3.fit(X_train__, y_train)
        y_pred__ = clf3.predict(X_test__)


        y_pred_proba_w_both = clf1.predict_proba(X_test)
        y_pred_proba_w_inp = clf2.predict_proba(X_test_)
        y_pred_proba_w_exp = clf3.predict_proba(X_test__)



        comprehensiveness, sufficiency = 0,0

        counter = 0
        for pred_proba_w_both, pred_proba_w_inp, pred_proba_w_exp, pred_class in zip(y_pred_proba_w_both,y_pred_proba_w_inp,y_pred_proba_w_exp,y_pred):
            comprehensiveness += (pred_proba_w_both[pred_class] - pred_proba_w_inp[pred_class])
            sufficiency += (pred_proba_w_both[pred_class] - pred_proba_w_exp[pred_class])
            counter+=1


        comprehensiveness = comprehensiveness / counter
        sufficiency = sufficiency / counter

        C.append(comprehensiveness)
        S.append(sufficiency)





        l, nl = 0,0
        us = 0
        for i ,j in zip(y_pred__,y_test):
            if i==j:
                l+=1
            else:
                nl+=1
        a,b=0,0
        for i,j,k,x in zip(y_pred,y_pred_,y_test,y_pred__):
            us+=int(i==k) - int(j==k)
            if k==x:
                a += int(i==k) - int(j==k)
            elif k!=x:
                b += int(i==k) - int(j==k)

        #print('LAS ',.5*(a/l)+.5*(b/nl))
        #print('Leakage Unadjusted Simulatability (LUS)', (us/(l+nl)))
        vals_lus.append(us/(l+nl))
        #print('Comprehensiveness ', comprehensiveness)
        #print('Sufficiency ', sufficiency)
        #print('f1 between prediction of proposed model and SVM w/ both inp and exp {}'.format(f1_score(y_test, y_pred, average='macro')))
        #print('f1 between prediction of proposed model and SVM w/ only inp {}'.format(f1_score(y_test, y_pred_, average='macro')))
        #print('f1 between prediction of proposed model and SVM w/ only exp {}'.format(f1_score(y_test, y_pred__, average='macro')))
        vals.append(.5*(a/l)+.5*(b/nl))
        k1.append(f1_score(y_test, y_pred, average='macro'))
        k2.append(f1_score(y_test, y_pred_, average='macro'))
        k3.append(f1_score(y_test, y_pred__, average='macro'))


    #print('Intra mean ', np.mean(ID), 'Intra std ', np.std(ID))
    #print('Inter mean ', np.mean(ID1), 'Inter std ', np.std(ID1))
    print('LAS mean ', np.mean(vals), 'LAS std ', np.std(vals))
    print('LUS mean ', np.mean(vals_lus), 'LUS std ', np.std(vals_lus))
    print('Comprehensiveness mean ', np.mean(C), 'Comprehensiveness std ', np.std(C))
    print('Sufficiency mean ', np.mean(S), 'Sufficiency std ', np.std(S))
    print('Avg. f1 between prediction of proposed model and SVM w/ both inp and exp {}'.format(np.mean(k1)))
    print('Avg. f1 between prediction of proposed model and SVM w/ only inp {}'.format(np.mean(k2)))
    print('Avg. f1 between prediction of proposed model and SVM w/ only exp {}'.format(np.mean(k3)))


In [ ]:
#R
cd = {i:[] for i in concept_classes}
import shutup; shutup.please()
def call():
    import math
    # Quantitative case study (1) Faithfulness on concepts
    # Quantitative case study (2) Are they explainable?
    test_dataloader=DataLoader(t_p, shuffle=True, batch_size=1)

    pred_r1, pred_r2 = [],[]
    pred_both = []
    act = []

    multimodal_repr = []
    concept_repr = []

    concept_repr1 = []
    concept_repr2 = []
    y_p = []

    cr, rr = [],[]
    cr1,rr1 = [],[]


    ours_icace = []
    ia_icace = []

    counter = 0


    c_p=0
    c_r=0

    a_p=0
    a_r=0

    average_precision_causal = 0
    average_precision_attrib = 0

    concept_cat = {}
    for i,c in enumerate(concept_classes):
        concept_cat[c] = all_concepts[i]

    for ii, batch in tqdm(enumerate(eval_dataloader)):

        if ii==0:
            print(ig)

        #if batch[1].detach().cpu().numpy()[0] == 1:

        ll = batch[1].detach().cpu().numpy()[0]
        if ll==1:

            #ll = batch[1].detach().cpu().numpy()[0]
            #print(ll, ii)
            y_p.append(ll)

            #print('*'*10)

            input_ids = batch[0]['input_ids'].to('cuda')
            #print(tokenizer.batch_decode(batch[0]['input_ids'], skip_special_tokens=True))
            token_type_ids = batch[0]['token_type_ids'].to('cuda')
            attention_mask = batch[0]['attention_mask'].to('cuda')
            visual_embeds = batch[0]['visual_embeds'].to('cuda')
            visual_token_type_ids = batch[0]['visual_token_type_ids'].to('cuda')
            visual_attention_mask = batch[0]['visual_attention_mask'].to('cuda')

            inputs_embeds = vb_wrapper.vb.vb.embeddings.word_embeddings(input_ids)

            # print(inputs_embeds)
            #     (input1_attr, input2_attr), delta = ig.attribute(inputs = (all_concepts.to('cuda'), inputs_embeds), additional_forward_args = (attention_mask,
            #         token_type_ids,
            #         None,
            #         None,
            #         None,
            #         visual_embeds,
            #         visual_attention_mask,
            #         visual_token_type_ids), return_convergence_delta=True)

            #print(visual_embeds)

            #     (input1_attr, input2_attr, input3_attr) = ig.attribute(inputs = (all_concepts.unsqueeze(0).to('cuda'), inputs_embeds, visual_embeds), additional_forward_args = (attention_mask,
            #         token_type_ids,
            #         None,
            #         None,
            #         None,
            #         visual_attention_mask,
            #         visual_token_type_ids))
            #vb(input_ids = input_ids, token_type_ids = token_type_ids, attention_mask = attention_mask, visual_embeds=visual_embeds_latent, visual_attention_mask=visual_attention_mask_latent, visual_token_type_ids=visual_token_type_ids_latent)

            if type(ig) in [captum.attr._core.kernel_shap.KernelShap, captum.attr._core.input_x_gradient.InputXGradient, captum.attr._core.deep_lift.DeepLift, captum.attr._core.saliency.Saliency, captum.attr._core.integrated_gradients.IntegratedGradients, captum.attr._core.feature_ablation.FeatureAblation]:
                    baseline = None
                    #print('none')
            else:
                baseline = baselines
                #print('baseline')

            if DECONFOUND:
                all_conc = get_inlp(all_concepts).to('cuda')
            else:
                all_conc = all_concepts.to('cuda')

            #print(all_conc)
            #print(all_concepts)

            #print(0/0)

            #all_conc = all_concepts.to('cuda')

            if type(ig) in [captum.attr._core.input_x_gradient.InputXGradient,captum.attr._core.saliency.Saliency]:
                #print('ip_grad')
                (input1_attr, input2_attr, input3_attr) = ig.attribute(inputs = (all_conc.unsqueeze(0).to('cuda'), inputs_embeds, visual_embeds),

                                                                    additional_forward_args = (attention_mask,
                token_type_ids,
                None,
                None,
                None,
                visual_attention_mask,
                visual_token_type_ids))

            else:
                #print('others')

                (input1_attr, input2_attr, input3_attr) = ig.attribute(inputs = (all_conc.unsqueeze(0).to('cuda'), inputs_embeds, visual_embeds),
                                                                      baselines = baseline,
                                                                        additional_forward_args = (attention_mask,
                    token_type_ids,
                    None,
                    None,
                    None,
                    visual_attention_mask,
                    visual_token_type_ids))


            #print(input1_attr, input2_attr)
            #print(input1_attr.shape, input2_attr.shape)
            k1=input1_attr.sum(dim=-1).squeeze(0).detach().cpu().numpy()

            conc_scr = []
            attribution_score = {}
            for i in range(18):
                cc = concept_classes[i]
                #print('relative attribution scores for concept {} is {}'.format(cc,k[i]))
                conc_scr.append((cc,k1[i]))
                attribution_score[cc] = k1[i]
                #conc_scr[cc] = k1[i]
            conc_scr.sort(key=lambda tup: tup[1], reverse=True)

            d = {}

            for cnt,i in enumerate(conc_scr):
                d[i[0]] = cnt

            #print(d)


            orig,pooled_op = model_pred(input_ids,attention_mask,token_type_ids,visual_embeds,visual_attention_mask,visual_token_type_ids,all_concepts)
            orig = orig[0]
            pooled_op = pooled_op.detach().cpu()
            multimodal_repr.append(pooled_op)
            #print('predicted class {}'.format(class_map[np.argmax(orig.detach().cpu().numpy())]))
            #print('actual class {}'.format(ll))
            #print(0/0)
            #orig = orig[0]
            k = []
            pred_class = np.argmax(orig.detach().cpu().numpy())
            causal_score = {}
            #print(pred_class)
            for i in range(18):
                #print('Removing concept: {}'.format(concept_classes[i]))
                cc = concept_classes[i]
                after_intervention,_ = model_pred(input_ids,attention_mask,token_type_ids,visual_embeds,visual_attention_mask,visual_token_type_ids,all_concepts,concept_no=i)
                after_intervention = after_intervention[0][pred_class]
                #print('hello')
                #print(orig[pred_class], after_intervention)
                #print(0/0)
                k.append((cc,abs(after_intervention-orig[pred_class]).item()))
                causal_score[cc] = abs(after_intervention-orig[pred_class]).item()
                #print('model prediction after removal {}'.format(model_pred(input_ids,attention_mask,token_type_ids,visual_embeds,visual_attention_mask,visual_token_type_ids,concept_no=i)))
            #k = np.asarray(k)
            #k = np.asarray(k)
            #k = list(map(lambda x: ((x-np.mean(k))/np.mean(k))*100, k))
            k.sort(key=lambda tup: tup[1], reverse=True)

            #print(k)

            #random.seed(counter)
            #random.shuffle(k)

            #print(k)

            #print(0/0)


            for i in k:
                cd[i[0]].append(i[1])



            #print(list(map(lambda x: x[0], list(k))))

            gamma = 0.9
            temp = []
            for cnt,i in enumerate(list(map(lambda x: x[0], list(k)))):
                temp.append((gamma**cnt) * concept_cat[i])
                #temp.append((math.e**(-cnt)) * concept_cat[i])
                #temp.append((1.1**(-cnt)) * concept_cat[i])

            temp = torch.stack(temp).mean(dim=0).unsqueeze(dim=0)
            #print(temp)
            #print(temp.shape)

            concept_repr.append(temp)



            temp = []
            for cnt,i in enumerate(list(map(lambda x: x[0], list(conc_scr)))):
                temp.append((gamma**cnt) * concept_cat[i])
                #temp.append((math.e**(-cnt)) * concept_cat[i])
                #temp.append((1.1**(-cnt)) * concept_cat[i])

            temp = torch.stack(temp).mean(dim=0).unsqueeze(dim=0)
            #print(temp)
            #print(temp.shape)

            concept_repr1.append(temp)


            #print(list(map(lambda x: x[0], list(conc_scr))))
            #print(0/0)

            #considering the k to be the causally sorted rank, we estimate ranks of

            ip_attrib_causal = []



            for i in range(18):
                cc = concept_classes[i]
                ip_attrib_causal.append((cc,((2*attribution_score[cc]*causal_score[cc]) / (attribution_score[cc]+causal_score[cc]))))



            ip_attrib = []
            for i in range(18):
                cc = concept_classes[i]
                ip_attrib.append((cc,attribution_score[cc]))





            ip_attrib_causal.sort(key=lambda tup: tup[1], reverse=True)
            ip_attrib.sort(key=lambda tup: tup[1], reverse=True)



            ra_icace_ours = []
            for cnt,i in enumerate(list(map(lambda x: x[0], list(k)))):
                ra_icace_ours.append((gamma**cnt) * causal_score[i])
                #ra_icace_ours.append((math.e**(-cnt)) * causal_score[i])
                #ra_icace_ours.append((1.1**(-cnt)) * causal_score[i])

            ra_icace_ia = []
            for cnt,i in enumerate(list(map(lambda x: x[0], list(conc_scr)))):
                ra_icace_ia.append((gamma**cnt) * causal_score[i])
                #ra_icace_ia.append((math.e**(-cnt)) * causal_score[i])
                #ra_icace_ia.append((1.1**(-cnt)) * causal_score[i])



            ours_icace.append(np.mean(ra_icace_ours))
            ia_icace.append(np.mean(ra_icace_ia))




            temp = []
            for cnt,i in enumerate(list(map(lambda x: x[0], list(ip_attrib_causal)))):
                temp.append((gamma**cnt) * concept_cat[i])
                #temp.append((math.e**(-cnt)) * concept_cat[i])
                #temp.append((1.1**(-cnt)) * concept_cat[i])

            temp = torch.stack(temp).mean(dim=0).unsqueeze(dim=0)

            concept_repr2.append(temp)



            d1 = {}

            for cnt,i in enumerate(ip_attrib_causal):
                d1[i[0]] = cnt

            r1,r2,r3 = [],[],[]
            for cnt,i in enumerate(list(map(lambda x: x[0], list(k)))):
                r1.append(cnt)
                r2.append(d[i])
                r3.append(d1[i])

            #print(r1,r2)
            corr, _ = stats.kendalltau(r1, r2)
            res,_ = stats.spearmanr(r1, r2)

            corr1, _ = stats.kendalltau(r1, r3)
            res1,_ = stats.spearmanr(r1, r3)

            #print('kendall tau', corr)
            cr.append(corr)
            #print('spearmans rho', res)
            rr.append(res)

            cr1.append(corr1)
            rr1.append(res1)


            rel_items = gc[counter]
            causal = [i[0] for i in k]
            #causal = random.sample(concept_classes,5)
            attrib = [i[0] for i in ip_attrib]


            #         mrr_causal = []
            #         for i in range(len(causal)):
            #             item = causal[i]
            #             if item in rel_items:
            #                 mrr_causal.append(1/(i+1))

            #         mrr_attrib = []
            #         for i in range(len(attrib)):
            #             item = attrib[i]
            #             if item in rel_items:
            #                 mrr_attrib.append(1/(i+1))



            #         mrr_causal = np.mean(np.asarray(mrr_causal))

            #print('gc', rel_items)
            #print('causal', causal)
            #print('attrib', attrib)

            K = 5

            rel_rec_items_at_K = set(causal[:K]) & set(rel_items)
            rec_items_at_K = causal[:K]
            tot_rel_items = len(rel_items)

            #print('Causal P@{} '.format(K), len(rel_rec_items_at_K) / len(rec_items_at_K) )
            #print('Causal R@{} '.format(K), len(rel_rec_items_at_K) / tot_rel_items )

            c_p+=len(rel_rec_items_at_K) / len(rec_items_at_K)
            c_r+=len(rel_rec_items_at_K) / tot_rel_items


            rel_rec_items_at_K = set(attrib[:K]) & set(rel_items)
            rec_items_at_K = attrib[:K]
            tot_rel_items = len(rel_items)

            #print('Attrib P@{} '.format(K), len(rel_rec_items_at_K) / len(rec_items_at_K) )
            #print('Attrib R@{} '.format(K), len(rel_rec_items_at_K) / tot_rel_items )

            a_p+=len(rel_rec_items_at_K) / len(rec_items_at_K)
            a_r+=len(rel_rec_items_at_K) / tot_rel_items


            tot_precision_causal = []
            for k in range(1,K+1):
                tot_precision_causal.append(len(set(causal[:k]) & set(rel_items)) / len(causal[:k]))

            tot_precision_attrib = []
            for k in range(1,K+1):
                tot_precision_attrib.append(len(set(attrib[:k]) & set(rel_items)) / len(attrib[:k]))



            average_precision_causal += np.mean(np.asarray(tot_precision_causal))
            average_precision_attrib += np.mean(np.asarray(tot_precision_attrib))

            counter+=1













    multimodal_repr = torch.stack(multimodal_repr).squeeze(1)

    #causal
    concept_repr = torch.stack(concept_repr).squeeze(1)


    #attribution
    concept_repr1 = torch.stack(concept_repr1).squeeze(1)

    #HM
    concept_repr2 = torch.stack(concept_repr2).squeeze(1)

    print('mean precision causal@5',c_p/counter)
    print('mean recall causal@5', c_r/counter)

    print('mean precision attrb@5',a_p/counter)
    print('mean recall attrb@5', a_r/counter)

    print('MAP@5 Causal ', average_precision_causal/counter)
    print('MAP@5 Attrib ', average_precision_attrib/counter)

    #     print('kendall tau', np.mean(cr))

    #     print('spearmans rho', np.mean(rr))


    #     print('-----------CAUSAL----------')
    #     get_sim(multimodal_repr, concept_repr)

    #     print('-----------Attribution----------')
    #     get_sim(multimodal_repr, concept_repr1)














In [ ]:

baire = ['ig', 'saliency', 'deeplift', 'deepliftshap', 'gradshap', 'ipxgrad']
#R
#baire = ['ig', 'saliency']
import captum
from scipy import stats
for fu in range(len(baire)):


    vb_wrapper = VB_wrapper(vb)
    vb_wrapper.eval()
    vb_wrapper.zero_grad()

    if baire[fu]=='ig':
        ig = IntegratedGradients(vb_wrapper)
    elif baire[fu]=='saliency':
        ig = Saliency(vb_wrapper)
    elif baire[fu]=='deeplift':
        ig = DeepLift(vb_wrapper)
    elif baire[fu]=='deepliftshap':
        ig = DeepLiftShap(vb_wrapper)
    elif baire[fu]=='gradshap':
        ig = GradientShap(vb_wrapper)
    elif baire[fu]=='ipxgrad':
        ig = InputXGradient(vb_wrapper)

    cr, rr, multimodal_repr, concept_repr, concept_repr1, y_p = call_sim()
    print('kendall tau', cr)

    print('spearmans rho', rr)


    print('-----------CAUSAL----------')
    get_sim(multimodal_repr, concept_repr, y_p)

    print('-----------Attribution----------')
    get_sim(multimodal_repr, concept_repr1, y_p)

660it [31:01,  2.82s/it]


kendall tau 0.017468805704099824
spearmans rho 0.025549613784907905
-----------CAUSAL----------
(660, 1536)
(660, 768)
(660, 768)
[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0,

660it [28:29,  2.59s/it]


kendall tau 0.09001782531194297
spearmans rho 0.12701003846514683
-----------CAUSAL----------
(660, 1536)
(660, 768)
(660, 768)
[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1

660it [28:36,  2.60s/it]


...

kendall tau -0.003188750247573776
spearmans rho -0.004778434499796731
-----------CAUSAL----------
(660, 1536)
(660, 768)
(660, 768)
[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 

660it [29:59,  2.73s/it]


kendall tau 0.013250148544266193
spearmans rho 0.01847577946649154
-----------CAUSAL----------
(660, 1536)
(660, 768)
(660, 768)
[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 

660it [29:24,  2.67s/it]


kendall tau 0.0002574767280649636
spearmans rho 0.00015636238546455316
-----------CAUSAL----------
(660, 1536)
(660, 768)
(660, 768)
[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,